Imports

In [11]:
from tensorflow import keras
import tensorflow as tf
from keras import layers
from keras.datasets.cifar import load_batch
from keras import backend
import numpy as np
import os
import datetime
from tensorflow.keras import regularizers



#Dataset Load

In [12]:
path = "datasets"
num_train_samples = 50000
x_train = np.empty((num_train_samples, 3, 32, 32), dtype="uint8")
y_train = np.empty((num_train_samples,), dtype="uint8")
for i in range(1, 6):
    fpath = os.path.join(path, "data_batch_" + str(i))
    (x_train[(i - 1) * 10000 : i * 10000, :, :, :],y_train[(i - 1) * 10000 : i * 10000],) = load_batch(fpath)
fpath = os.path.join(path, "test_batch")
x_test, y_test = load_batch(fpath)
y_train = np.reshape(y_train, (len(y_train), 1))
y_test = np.reshape(y_test, (len(y_test), 1))
if backend.image_data_format() == "channels_last":
    x_train = x_train.transpose(0, 2, 3, 1)
    x_test = x_test.transpose(0, 2, 3, 1)
x_test = x_test.astype(x_train.dtype)
y_test = y_test.astype(y_train.dtype)
print(x_train.shape,x_test.shape)
# (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()


(50000, 32, 32, 3) (10000, 32, 32, 3)


In [13]:
num_classes = 10
img_rows, img_cols = 32, 32

Normalise

In [14]:
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

In [15]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Model definition

In [16]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation="relu",
          input_shape=(img_rows, img_cols, 3)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu"))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation="relu",
          kernel_regularizer=regularizers.l2(0.001)))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))
model.add(tf.keras.layers.ZeroPadding2D(padding=((1, 1), (1, 1))))
model.add(tf.keras.layers.Conv2D(128, (3, 3), activation="relu",
          kernel_regularizer=regularizers.l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dense(64, activation="relu"))
model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))


model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1)


In [17]:
# Load the TensorBoard notebook extension
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [18]:
model.fit(x_train, y_train, epochs=25, batch_size=64,
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Epoch 1/25
782/782 [==============================] - 26s 32ms/step - loss: 1.7855 - accuracy: 0.3444 - val_loss: 1.4331 - val_accuracy: 0.5024
Epoch 2/25
782/782 [==============================] - 23s 29ms/step - loss: 1.3831 - accuracy: 0.5184 - val_loss: 1.3267 - val_accuracy: 0.5405
Epoch 3/25
782/782 [==============================] - 23s 29ms/step - loss: 1.2305 - accuracy: 0.5851 - val_loss: 1.2400 - val_accuracy: 0.5797
Epoch 4/25
782/782 [==============================] - 22s 29ms/step - loss: 1.1291 - accuracy: 0.6281 - val_loss: 1.1442 - val_accuracy: 0.6171
Epoch 5/25
782/782 [==============================] - 23s 29ms/step - loss: 1.0569 - accuracy: 0.6563 - val_loss: 1.1325 - val_accuracy: 0.6250
Epoch 6/25
782/782 [==============================] - 22s 29ms/step - loss: 1.0092 - accuracy: 0.6787 - val_loss: 1.0565 - val_accuracy: 0.6627
Epoch 7/25
782/782 [==============================] - 22s 28ms/step - loss: 0.9742 - accuracy: 0.6926 - val_loss: 1.0122 - val_accuracy:

In [19]:
%tensorboard --logdir logs/fit

# evaluate the model

In [20]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print("Test accuracy:", test_acc)

313/313 - 2s - loss: 0.9153 - accuracy: 0.7429 - 2s/epoch - 6ms/step
Test accuracy: 0.742900013923645


In [21]:
model_predictions = model.predict(x_test)

# Compute the mean of the validation set labels
mean_val_labels = np.mean(y_test)

# Compute the total variance of the validation set
total_variance = np.mean(np.square(y_test - mean_val_labels))

# Compute the bias squared
bias_squared = np.square(np.mean(y_test - model_predictions))

# Compute the variance of the model predictions
variance = total_variance - bias_squared

print("Bias squared:", bias_squared)
print("Variance:", variance)

313/313 [==============================] - 2s 6ms/step
Bias squared: 3.1443736e-22
Variance: 0.09


Saving the Keras model

In [22]:
model.save('saved_model/Classification')




INFO:tensorflow:Assets written to: saved_model/Classification\assets


INFO:tensorflow:Assets written to: saved_model/Classification\assets


converting and saving the Lite version of the Model  

In [23]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
with open('saved_model/LiteModel.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\zizou\AppData\Local\Temp\tmp4qashltr\assets


INFO:tensorflow:Assets written to: C:\Users\zizou\AppData\Local\Temp\tmp4qashltr\assets
